In [ ]:
# Finds the price of anarchy of a two-player game (utility of optimal play vs. Nash equilibrium)

In [3]:
import pandas as pd
import numpy as np

In [4]:
# Define player class
class Player:
    def __init__(self, playerNum, payoffMatrix):
        self.playerNum    = playerNum
        self.payoffMatrix = payoffMatrix

In [5]:
Player.name         = 'Player1'
Player.payoffMatrix = [[1,2],[3,4]]

In [6]:
Player1 = Player('Player1', [[3,0],[5,1]])
Player2 = Player('Player2', [[3,5],[0,1]])

In [7]:
Player1.payoffMatrix
Player2.payoffMatrix

[[3, 5], [0, 1]]

In [67]:
### Find pure strategy Nash equilibrium by iteratively searching whether there's improvement in each quadrant

In [88]:
# Calculate distance between each quadrant
# Size is rows/columns in the payoff matrix
size = 2
x = np.arange(0, size, 1)
y = np.arange(0, size, 1)
coordArr = [[(0,0) for j in y] for i in x]

In [89]:
coordArr

[[(0, 0), (0, 0)], [(0, 0), (0, 0)]]

In [71]:
for rowInd, row in enumerate((coordArr)):
    for colInd in range(len(row)):
        coordArr[rowInd][colInd] = ( rowInd, colInd )

In [82]:
coordArr

array([[[0, 0],
        [0, 0]],

       [[0, 0],
        [0, 0]]])

In [73]:
for pos in coordArr:
    print(pos)

[(0, 0), (0, 1)]
[(1, 0), (1, 1)]


In [10]:
payoffs = Player1.payoffMatrix
for rowInd, row in enumerate((payoffs)):
    print(row)
    for colInd in range(len(row)):
        print(colInd)

[3, 0]
0
1
[5, 1]
0
1


In [11]:
payoffs

[[3, 0], [5, 1]]

In [12]:
range(len(payoffs))

range(0, 2)

In [13]:
(0,1) - (1,1)

TypeError: unsupported operand type(s) for -: 'tuple' and 'tuple'

In [15]:
np.subtract((0,1), (1,1))

array([-1,  0])